In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "9"

from typing import NamedTuple

import numpyro 
import numpyro.distributions as dist

from numpyro.handlers import condition, reparam, seed, trace
from numpyro.infer.reparam import LocScaleReparam, TransformReparam

In [29]:
from math import pi
from tqdm import tqdm

from typing import Optional, Tuple, Callable, Union, List
from functools import partial

import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from jax import vmap, jit, pmap
from jax import random
import optax

# jax.config.update("jax_enable_x64", True)

import numpy as np
from astropy import units as u
from astropy import constants as c


import odisseo
from odisseo import construct_initial_state
from odisseo.integrators import leapfrog
from odisseo.dynamics import direct_acc, DIRECT_ACC, DIRECT_ACC_LAXMAP, DIRECT_ACC_FOR_LOOP, DIRECT_ACC_MATRIX
from odisseo.option_classes import SimulationConfig, SimulationParams, MNParams, NFWParams, PlummerParams, MN_POTENTIAL, NFW_POTENTIAL
from odisseo.initial_condition import Plummer_sphere, ic_two_body, sample_position_on_sphere, inclined_circular_velocity, sample_position_on_circle, inclined_position
from odisseo.utils import center_of_mass
from odisseo.time_integration import time_integration
from odisseo.units import CodeUnits
from odisseo.visualization import create_3d_gif, create_projection_gif, energy_angular_momentum_plot
from odisseo.potentials import MyamotoNagai, NFW
from odisseo.option_classes import DIFFRAX_BACKEND, DOPRI5, TSIT5, SEMIIMPLICITEULER, LEAPFROGMIDPOINT, REVERSIBLEHEUN


plt.rcParams.update({
    'font.size': 20,
    'axes.labelsize': 20,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 15,
})




In [3]:
from functools import partial
import haiku as hk
from tensorflow_probability.substrates import jax as tfp
from jaxopt import Bisection
from jaxopt.linear_solve import solve_normal_cg

# tfp = tfp.substrates.jax
tfb = tfp.bijectors
tfd = tfp.distributions


# This module is to store our implicit inverse functions
@partial(jax.custom_vjp, nondiff_argnums=(0,))
def root_bisection(f, params):
    """
    f: optimality fn with input arg (params, x)
    """
    bisec = Bisection(
        optimality_fun=f,
        lower=0.0,
        upper=1.0,
        check_bracket=False,
        maxiter=100,
        tol=1e-06,
    )
    return bisec.run(None, params).params


def root_bisection_fwd(f, params):
    z_star = root_bisection(f, params)
    return z_star, (params, z_star)


def root_bwd(f, res, z_star_bar):
    params, z_star = res
    _, vjp_a = jax.vjp(lambda p: f(z_star, p), params)
    _, vjp_z = jax.vjp(lambda z: f(z, params), z_star)
    return vjp_a(solve_normal_cg(lambda u: vjp_z(u)[0], -z_star_bar))


root_bisection.defvjp(root_bisection_fwd, root_bwd)


def make_inverse_fn(f):
    """Defines the inverse of the input function, and provides implicit gradients
    of the inverse.

    Args:
      f: callable of input shape (params, x)
    Retuns:
      inv_f: callable of with args (params, y)
    """

    def inv_fn(params, y):
        def optimality_fn(x, params):
            p, y = params
            return f(p, x) - y

        return root_bisection(optimality_fn, [params, y])

    return inv_fn

# Bijiector functions
class MixtureAffineSigmoidBijector(tfp.bijectors.Bijector):
    """
    Bijector based on a ramp function, and implemented using an implicit
    layer.
    This implementation is based on the Smooth Normalizing Flows described
    in: https://arxiv.org/abs/2110.00351
    """

    def __init__(self, a, b, c, p, name="MixtureAffineSigmoidBijector"):
        """
        Args:
          rho: function of x that defines a ramp function between 0 and 1
          a,b,c: scalar parameters of the coupling layer.
        """
        super(self.__class__, self).__init__(forward_min_event_ndims=0, name=name)
        self.a = a
        self.b = b
        self.c = c
        self.p = p

        def sigmoid(x, a, b, c):
            z = (jax.scipy.special.logit(x) + b) * a
            y = jax.nn.sigmoid(z) * (1 - c) + c * x
            return y

        # Rescaled bijection
        def f(params, x):
            a, b, c, p = params
            a_in, b_in = [0.0 - 1e-1, 1.0 + 1e-1]

            x = (x - a_in) / (b_in - a_in)
            x0 = (jnp.zeros_like(x) - a_in) / (b_in - a_in)
            x1 = (jnp.ones_like(x) - a_in) / (b_in - a_in)

            y = sigmoid(x, a, b, c)
            y0 = sigmoid(x0, a, b, c)
            y1 = sigmoid(x1, a, b, c)

            y = (y - y0) / (y1 - y0)
            return jnp.sum(p * (y * (1 - c) + c * x), axis=0)

        self.f = f

        # Inverse bijector
        self.inv_f = make_inverse_fn(f)

    def _forward(self, x):
        return jax.vmap(jax.vmap(self.f))([self.a, self.b, self.c, self.p], x)

    def _inverse(self, y):
        return jax.vmap(jax.vmap(self.inv_f))([self.a, self.b, self.c, self.p], y)

    def _forward_log_det_jacobian(self, x):
        def logdet_fn(x, a, b, c, p):
            g = jax.grad(self.f, argnums=1)([a, b, c, p], x)
            s, logdet = jnp.linalg.slogdet(jnp.atleast_2d(g))
            return s * logdet

        return jax.vmap(jax.vmap(logdet_fn))(x, self.a, self.b, self.c, self.p)


#Coupling layers
class AffineCoupling(hk.Module):
    def __init__(
        self, y, *args, layers=[128, 128], activation=jax.nn.leaky_relu, **kwargs
    ):
        """
        Args:
        y, conditioning variable
        layers, list of hidden layers
        activation, activation function for hidden layers
        """
        self.y = y
        self.layers = layers
        self.activation = activation
        super().__init__(*args, **kwargs)

    def __call__(self, x, output_units, **condition_kwargs):
        net = jnp.concatenate([x, self.y], axis=-1)
        for i, layer_size in enumerate(self.layers):
            net = self.activation(hk.Linear(layer_size, name="layer%d" % i)(net))

        shifter = tfb.Shift(hk.Linear(output_units)(net))
        scaler = tfb.Scale(jnp.clip(jnp.exp(hk.Linear(output_units)(net)), 1e-2, 1e2))
        return tfb.Chain([shifter, scaler])


class AffineSigmoidCoupling(hk.Module):
    """This is the coupling layer used in the Flow."""

    def __init__(
        self,
        y,
        *args,
        layers=[128, 128],
        n_components=32,
        activation=jax.nn.silu,
        **kwargs
    ):
        """
        Args:
        y, conditioning variable
        layers, list of hidden layers
        n_components, number of mixture components
        activation, activation function for hidden layers
        """
        self.y = y
        self.layers = layers
        self.n_components = n_components
        self.activation = activation
        super().__init__(*args, **kwargs)

    def __call__(self, x, output_units, **condition_kwargs):
        net = jnp.concatenate([x, self.y], axis=-1)
        for i, layer_size in enumerate(self.layers):
            net = self.activation(hk.Linear(layer_size, name="layer%d" % i)(net))

        log_a_bound = 4
        min_density_lower_bound = 1e-4
        n_components = self.n_components

        log_a = (
            jax.nn.tanh(hk.Linear(output_units * n_components, name="l3")(net))
            * log_a_bound
        )
        b = hk.Linear(output_units * n_components, name="l4")(net)
        c = min_density_lower_bound + jax.nn.sigmoid(
            hk.Linear(output_units * n_components, name="l5")(net)
        ) * (1 - min_density_lower_bound)
        p = hk.Linear(output_units * n_components, name="l6")(net)

        log_a = log_a.reshape(-1, output_units, n_components)
        b = b.reshape(-1, output_units, n_components)
        c = c.reshape(-1, output_units, n_components)
        p = p.reshape(-1, output_units, n_components)
        p = jax.nn.softmax(p)

        return MixtureAffineSigmoidBijector(jnp.exp(log_a), b, c, p)

# Normalizing flow model 
class ConditionalRealNVP(hk.Module):
    """A normalizing flow based on RealNVP using specified bijector functions."""

    def __init__(
        self, d, *args, n_layers=3, bijector_fn=AffineSigmoidCoupling, **kwargs
    ):
        """
        Args:
        d, dimensionality of the input
        n_layers, number of layers
        coupling_layer, list of coupling layers
        """
        self.d = d
        self.n_layer = n_layers
        self.bijector_fn = bijector_fn
        super().__init__(*args, **kwargs)

    def __call__(self, y):
        chain = tfb.Chain(
            [
                tfb.Permute(jnp.arange(self.d)[::-1])(
                    tfb.RealNVP(
                        self.d // 2, bijector_fn=self.bijector_fn(y, name="b%d" % i)
                    )
                )
                for i in range(self.n_layer)
            ]
        )

        nvp = tfd.TransformedDistribution(
            tfd.MultivariateNormalDiag(0.5 * jnp.ones(self.d), 0.05 * jnp.ones(self.d)),
            bijector=chain,
        )

        return nvp


2025-04-29 15:29:02.157045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745933342.175765 2723962 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745933342.181166 2723962 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745933342.195903 2723962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745933342.195930 2723962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745933342.195932 2723962 computation_placer.cc:177] computation placer alr

In [25]:
from typing import NamedTuple

#mimic the argument parser used in the sbi_bm_lens

class args_namedtuple(NamedTuple):

    #seed
    seed = 42

    #activation function
    activ_fun = "silu"

    #Normalizing flow
    nf = "smooth"

    #score type
    score_type = "conditional"

    #sbi method
    sbi_method = "nle"

    #number of bijiector layers
    n_bijector_layers = 5

    #number of coupling layers
    n_flow_layers = 3

    #number of parameters
    n_params = 5 #["t_end", "Mtot_plummer", "a_plummer", "M_NFW", "r_s"]

    #number of step
    total_steps = 10_000


args = args_namedtuple()

master_seed = jax.random.PRNGKey(args.seed)

In [14]:
#### CREATE THE NDE #####

#activation  function
if args.activ_fun == "silu":
    activ_fun = jax.nn.silu
elif args.activ_fun == "sin":
    activ_func = jnp.sin

#score type
if args.sbi_method == "nle":
    score_type = "conditional"  #need to be pass to the model object (stochastic simulator)

#NF type, either smooth or affine
if args.nf == "smooth":
    if args.sbi_method == "npe":
        pass
    elif args.sbi_method == "nle":
        # how these qantities are comute (with dataset form prior)
        # scale_y = jnp.std(dataset_y, axis=0) / 0.07
        # shift_y = jnp.mean(dataset_y / scale_y, axis=0) - 0.5

        #just place holder, need to be calculated from the dataset for normalization 
        scale = jnp.ones(args.n_params)
        shift = jnp.ones(args.n_params)

        #NF ARCHITECTURE
        #128 neuron per layer
        bijector_layers = [128] * args.n_bijector_layers
        
        bijector = partial(
            AffineSigmoidCoupling,
            layers=bijector_layers,
            activation=activ_fun,
            n_components=16,
        )
        
        NF = partial(ConditionalRealNVP, 
                     n_layers=args.n_flow_layers, 
                     bijector_fn=bijector)

        class NDE(hk.Module):
            def __call__(self, y):
                nvp = NF(args.n_params)(y)
                return tfd.TransformedDistribution(
                    nvp, tfb.Chain([tfb.Scale(scale), tfb.Shift(shift)])
                )
        
elif args.nf == "affine":
    bijector_layers = [128] * args.n_bijector_layers

    bijector = partial(AffineCoupling, layers=bijector_layers, activation=activ_fun)

    NF = partial(ConditionalRealNVP, n_layers=args.n_flow_layers, bijector_fn=bijector)

    class NDE(hk.Module):
        def __call__(self, y):
            return NF(args.n_params)(y)
        
#FOR NPE NEED TO HAVE SMOOTH NORMALIZING FLOW
if args.nf == "affine" and args.sbi_method == "npe" and args.score_weight > 0:
    raise ValueError("NDE has to be smooth")

elif args.sbi_method == "nle":
    nf_log_prob = hk.without_apply_rng(
        hk.transform(lambda theta, y: NDE()(theta).log_prob(y).squeeze())
    )


print("done ✓")


done ✓


In [19]:
#This should be used to create the compress dataset, in this simplified case
#we are using it just to sample the prior, to be implemented the compression

class JaxCompressedSimulator:
    def __init__(self, ):

        # self.compressor = compressor #not used
        # self.params_compressor = params_compressor #not used
        # self.opt_state = opt_state #not used

        self.t_end = dist.Uniform(0.5, 10.0)
        self.Mtot_plummer = dist.Uniform(1e3, 1e5)
        self.a_plummer = dist.Uniform(0.1, 2.0)
        self.M_NFW = dist.Uniform(5e11, 1.5e12)
        self.r_s = dist.Uniform(1.0, 20.0)

        self.stack = [
            self.t_end,
            self.Mtot_plummer,
            self.a_plummer,
            self.M_NFW,
            self.r_s,
        ]

    def prior_sample(self, sample_shape, master_key):
        samples = []

        keys = jax.random.split(master_key, 6)

        for i, distribution in enumerate(self.stack):
            samples.append(distribution.sample(keys[i], sample_shape))

        return jnp.stack(samples).T

    def prior_log_prob(self, values):
        logp = 0

        for i, distribution in enumerate(self.stack):
            logp += distribution.log_prob(values[..., i])

        return logp

compressed_simulator = JaxCompressedSimulator()

In [26]:
######## INFERENCE ########
print("... inference")

params_init = nf_log_prob.init(
    master_seed, 0.5 * jnp.ones([1, args.n_params]), 0.5 * jnp.ones([1, args.n_params])
)

prior_mean = jnp.mean(compressed_simulator.prior_sample((1000,), master_seed), axis=0)

nb_steps = args.total_steps - args.total_steps * 0.2

lr_scheduler = optax.exponential_decay(
    init_value=0.001,
    transition_steps=nb_steps // 50,
    decay_rate=0.9,
    end_value=1e-5,
)

... inference


In [ ]:
class SNLE:
    def __init__(self, NDE, init_params_nde, args):
        self.NDE = NDE
        self.params = init_params_nde
        self.dim = args.n_params

    def log_prob_fn(self, params, theta, y):
        return self.NDE.apply(params, theta, y)

    def loss_nll_and_score(self, params, mu, batch, score, weight_score):
        lp, out = jax.vmap(
            jax.value_and_grad(
                lambda theta, x: self.log_prob_fn(
                    params, theta.reshape([1, self.dim]), x.reshape([1, self.dim])
                ).squeeze()
            )
        )(mu, batch)

        return (
            -jnp.mean(lp) + weight_score * jnp.sum((out - score) ** 2, axis=-1).mean()
        )

    def loss_nll(self, params, mu, batch, score, weight_score):
        lp = self.log_prob_fn(params, mu, batch)

        return -jnp.mean(lp)

    def train(
        self, data, learning_rate, total_steps=30_000, batch_size=128, score_weight=0
    ):
        dataset_theta = data["theta"]
        dataset_y = data["y"]
        if score_weight != 0:
            dataset_score = data["score"]
            loss_fn = self.loss_nll_and_score
        else:
            loss_fn = self.loss_nll

        nb_simu = len(dataset_theta)

        print("nb of simulations used for training: ", nb_simu)

        params = self.params
        optimizer = optax.adam(learning_rate)
        opt_state = optimizer.init(params)

        @jax.jit
        def update(params, opt_state, mu, batch, score, weight_score):
            """Single SGD update step."""
            loss, grads = jax.value_and_grad(loss_fn)(
                params, mu, batch, score, weight_score
            )
            updates, new_opt_state = optimizer.update(grads, opt_state, params)
            new_params = optax.apply_updates(params, updates)

            return loss, new_params, new_opt_state

        print("... start training")

        batch_loss = []
        lr_scheduler_store = []
        pbar = tqdm(range(total_steps))

        for batch in pbar:
            inds = np.random.randint(0, nb_simu, batch_size)
            ex_theta = dataset_theta[inds]
            ex_y = dataset_y[inds]
            if score_weight != 0:
                ex_score = dataset_score[inds]
            else:
                ex_score = None

            if not jnp.isnan(ex_y).any():
                l, params, opt_state = update(
                    params, opt_state, ex_theta, ex_y, ex_score, score_weight
                )

                batch_loss.append(l)
                pbar.set_description(f"loss {l:.3f}")

                if jnp.isnan(l):
                    break

        self.params = params
        self.loss = batch_loss

        print("done ✓")

    def sample(
        self,
        log_prob_prior,
        observation,
        init_point,
        key,
        num_results=3e4,
        num_burnin_steps=5e2,
        num_chains=12,
    ):
        print("... running hmc")

        @jax.vmap
        def unnormalized_log_prob(theta):
            prior = log_prob_prior(theta)

            likelihood = self.log_prob_fn(
                self.params,
                theta.reshape([1, self.dim]),
                jnp.array(observation).reshape([1, self.dim]),
            )

            return likelihood + prior

        # Initialize the HMC transition kernel.
        adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.HamiltonianMonteCarlo(
                target_log_prob_fn=unnormalized_log_prob,
                num_leapfrog_steps=3,
                step_size=1e-2,
            ),
            num_adaptation_steps=int(num_burnin_steps * 0.8),
        )

        # Run the chain (with burn-in).
        # @jax.jit
        def run_chain():
            # Run the chain (with burn-in).
            samples, is_accepted = tfp.mcmc.sample_chain(
                num_results=num_results,
                num_burnin_steps=num_burnin_steps,
                current_state=jnp.array(init_point) * jnp.ones([num_chains, self.dim]),
                kernel=adaptive_hmc,
                trace_fn=lambda _, pkr: pkr.inner_results.is_accepted,
                seed=key,
            )

            return samples, is_accepted

        samples_hmc, is_accepted_hmc = run_chain()
        sample_nd = samples_hmc[is_accepted_hmc]

        print("done ✓")

        return sample_nd

In [ ]:
if args.sbi_method == "nle":
    inference = SNLE(NDE=nf_log_prob, init_params_nde=params_init, dim=6)